In [ ]:
import numpy as np
import pandas as pd
import urllib.request as ureq
import json
import csv

evic_table = pd.read_csv("data/Eviction_Notices.csv", low_memory=False)
url = "https://geo.fcc.gov/api/census/area?"
fips_arr = []

def parseLocation(ln):
    return ln[7:-1].split(" ")

In [28]:
import urllib.error as uerr
for i, row in evic_table.iterrows():
    loc = parseLocation(str(row['Location']))
    if (len(loc) > 1):
        try:
            res = json.loads(ureq.urlopen(url + "lon=" + loc[0] + "&lat=" + loc[1]).read())
        except uerr.HTTPError as e:
            fips_arr.append('')
            print("Error at " + str(row))
        else:
            fips = res['results'][0]['block_fips'][1:11]
            fips_arr.append(fips)
    else:
        fips_arr.append('')
        
evic_table['fips code'] = pd.Series(fips_arr)

In [27]:
import datetime as dt
evic_table['fips code'] = pd.Series(fips_arr)
for i, row in evic_table.iterrows():
    evic_table.at[i, "File Date"] = dt.datetime.strptime(row['File Date'],"%Y-%m-%d").strftime("%m-%d-%Y")
evic_table.to_csv("data/Evictions_fips.csv")

In [70]:
modified_evic = pd.read_csv("data/Evictions_fips.csv", low_memory=False)
with open("data/udp_2017.csv", mode='r') as infile:
    reader = csv.reader(infile)
    gen_dict = dict((str(rows[0]),rows[1]) for rows in reader)

In [81]:
# stats_dict = dict.fromkeys(modified_evic['fips code'], 0)
# x = []
# y = []

# for i, row in modified_evic.iterrows():
#     if not math.isnan(row['fips code']):
#         stats_dict[modified_evic.at[i, 'fips code']] += 1

# for i, row in modified_evic.iterrows():
#     if not math.isnan((row['fips code'])):
#         x.append(stats_dict[modified_evic.at[i, 'fips code']])
#         y.append(gen_dict[str(row['fips code'])[0:-2]])